# Extract CRPS for one run instead of all

In [1]:
%cd /home/ltchen/gnnpp
import sys
import os
import pytorch_lightning as L
import torch
import torch_geometric
import json
import copy

from typing import Tuple
from torch_geometric.nn import GATv2Conv
from torch_geometric.utils import scatter
from torch.nn import Linear, ModuleList, ReLU
from torch_geometric.loader import DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim import AdamW
from pytorch_lightning.loggers import WandbLogger

from models.loss import NormalCRPS
from models.model_utils import MakePositive, EmbedStations
from utils.data import (
    load_dataframes,
    summary_statistics,
)
from exploration.graph_creation import *
from models.graphensemble.multigraph import *
import shap
import matplotlib.pyplot as plt
from torch_geometric.explain import GNNExplainer

/home/ltchen/gnnpp


/home/ltchen/.conda/envs/gnn_env4/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
leadtime = "24h"
# data_type = "f"

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
DIRECTORY = os.getcwd()

In [3]:
dataframes = load_dataframes(leadtime=leadtime)
dataframes = summary_statistics(dataframes)

[INFO] Dataframes exist. Will load pandas dataframes.
[INFO] Calculating summary statistics for train
[INFO] Calculating summary statistics for valid
[INFO] Calculating summary statistics for test_rf
[INFO] Calculating summary statistics for test_f


In [18]:
graph_name = "g1"
drn = False
if drn == True:
    SAVEPATH = os.path.join(DIRECTORY, f"leas_trained_models/drn_{leadtime}/models")
    JSONPATH = os.path.join(DIRECTORY, f"trained_models/drn_{leadtime}/params.json")
else:
    SAVEPATH = os.path.join(DIRECTORY, f"leas_trained_models/sum_stats_{leadtime}/{graph_name}_{leadtime}/models")
    JSONPATH = os.path.join(DIRECTORY, f"leas_trained_models/sum_stats_{leadtime}/{graph_name}_{leadtime}/params.json")


with open(JSONPATH, "r") as f:
    print(f"[INFO] Loading {JSONPATH}")
    args_dict = json.load(f)
config = args_dict

[INFO] Loading /home/ltchen/gnnpp/leas_trained_models/sum_stats_24h/g1_24h/params.json


In [19]:
graphs1_train_rf, tests1 = normalize_features_and_create_graphs1(df_train=dataframes['train'], df_valid_test=[dataframes['valid'], dataframes['test_rf'], dataframes['test_f']], station_df=dataframes['stations'], attributes=["geo"], edges=[("geo", 50)], sum_stats = True)
graphs1_valid_rf, graphs1_test_rf, graphs1_test_f = tests1

g1_train_loader = DataLoader(graphs1_train_rf, batch_size=config['batch_size'], shuffle=True)
g1_valid_loader = DataLoader(graphs1_valid_rf, batch_size=config['batch_size'], shuffle=True)
g1_test_f_loader = DataLoader(graphs1_test_f, batch_size=config['batch_size'], shuffle=False)
g1_test_rf_loader = DataLoader(graphs1_test_rf, batch_size=config['batch_size'], shuffle=False)

train_loader = g1_train_loader
valid_loader = g1_valid_loader
test_f_loader = g1_test_f_loader
test_rf_loader = g1_test_rf_loader
test_loader = [test_f_loader, test_rf_loader]

emb_dim = 20
in_channels = graphs1_train_rf[0].x.shape[1] + emb_dim - 1
edge_dim = graphs1_train_rf[0].num_edge_features
# max_epochs = max_epoch_list[graph_name]
num_nodes = graphs1_train_rf[0].num_nodes

[INFO] Normalizing features...
fit_transform
transform 1
transform 2
transform 3


100%|██████████| 730/730 [00:02<00:00, 278.50it/s]


In [23]:
TRAINNAME = f"{graph_name}_{leadtime}_train_run5"
CKPT_PATH = os.path.join(SAVEPATH, TRAINNAME + '.ckpt')
# RESULTPATH = os.path.join(DIRECTORY, f"leas_trained_models/best_{leadtime}/best_{leadtime}_{graph_name}")
RESULTPATH = JSONPATH = os.path.join(DIRECTORY, f"leas_trained_models/sum_stats_{leadtime}/{graph_name}_{leadtime}")
print(RESULTPATH)

multigraph = Multigraph.load_from_checkpoint(
    CKPT_PATH,
    num_nodes=num_nodes,
    embedding_dim=emb_dim,
    edge_dim=edge_dim,
    in_channels=in_channels,
    hidden_channels_gnn=config['gnn_hidden'],
    out_channels_gnn=config['gnn_hidden'],
    num_layers_gnn=config['gnn_layers'],
    heads=config['heads'],
    hidden_channels_deepset=config['gnn_hidden'],
    optimizer_class=AdamW,
    optimizer_params=dict(lr=config['lr']),
)
multigraph.eval()
trainer = L.Trainer(log_every_n_steps=1, accelerator="gpu", devices=1, enable_progress_bar=True)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


/home/ltchen/gnnpp/leas_trained_models/sum_stats_24h/g1_24h


In [24]:
data_list = ["f", "rf"]
for data, tl in zip(data_list, test_loader):
    preds_list = []
    preds = trainer.predict(model=multigraph, dataloaders=[tl]) # 92 x 976 x 2 forecasts with mu and sigma of 122 stations
    #print(preds[0].shape) # 960 (8*120), 2
    print(preds)
    preds = torch.cat(preds, dim=0)
    preds_list.append(preds)

    targets = dataframes[f"test_{data}"][1]
    print(targets)
    targets = torch.tensor(targets.t2m.values) - 273.15
    print(targets)
    stacked = torch.stack(preds_list)
    final_preds = torch.mean(stacked, dim=0)

    res = multigraph.loss_fn.crps(final_preds, targets)
    print("#############################################")
    print("#############################################")
    print(f"final crps: {res.item()}")
    print("#############################################")
    print("#############################################")

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/home/ltchen/.conda/envs/gnn_env4/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Predicting DataLoader 0: 100%|██████████| 92/92 [00:00<00:00, 175.23it/s]
[tensor([[  5.4764,   0.9657],
        [  1.7079,   0.9821],
        [  5.8840,   0.9532],
        ...,
        [ -0.7263,   1.9671],
        [ -8.8814,   1.8565],
        [-10.0871,   1.8289]]), tensor([[  5.7836,   0.8441],
        [  5.4352,   0.7729],
        [  5.5527,   0.8298],
        ...,
        [ -5.0819,   0.9713],
        [-13.1010,   1.3736],
        [-13.6914,   1.5255]]), tensor([[ -2.4225,   1.4941],
        [ -2.9347,   1.4581],
        [ -0.7746,   1.2894],
        ...,
        [ -7.1093,   1.8757],
        [-13.1637,   2.0603],
        [-15.6291,   2.5015]]), tensor([[-1.9374,  0.9148],
        [-2.0641,  0.9014],
        [-1.7251,  0.8453],
        ...,
        [ 5.4315,  1.9747],
        [ 0.8426,  1.5357],
        [-0.6183,  1.4441]]), tensor([[ 7.3312,  0.8524],
        [ 8.6911,  0.7618],
        [ 6.4944,  0.8150],
        ...,
        [ 0.7751,  1.8825],
        [-5.7890,  1.9763],
    

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


            time  station_id     t2m
0     2017-01-01           0  278.65
1     2017-01-01           1  275.25
2     2017-01-01           2  279.75
3     2017-01-01           3  279.15
4     2017-01-01           4  275.05
...          ...         ...     ...
87595 2018-12-31         115  277.95
87596 2018-12-31         116  276.85
87597 2018-12-31         117  276.35
87598 2018-12-31         118  270.65
87599 2018-12-31         119  268.85

[87600 rows x 3 columns]
tensor([ 5.5000,  2.1000,  6.6000,  ...,  3.2000, -2.5000, -4.3000],
       dtype=torch.float64)
#############################################
#############################################
final crps: 0.6390619057955299
#############################################
#############################################


/home/ltchen/.conda/envs/gnn_env4/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Predicting DataLoader 0: 100%|██████████| 92/92 [00:00<00:00, 178.11it/s]
[tensor([[ 6.8663,  0.6354],
        [ 7.2250,  0.6844],
        [ 6.8326,  0.6132],
        ...,
        [-2.4051,  1.3186],
        [-5.2691,  1.5928],
        [-8.1560,  2.0423]]), tensor([[-2.3410,  0.7293],
        [-2.8659,  0.8506],
        [-2.4693,  0.7078],
        ...,
        [ 2.0494,  1.0951],
        [-1.0644,  0.8649],
        [-1.7243,  0.9147]]), tensor([[  1.3163,   1.6544],
        [  1.5473,   1.3487],
        [  2.1642,   1.5999],
        ...,
        [ -4.0304,   0.7759],
        [-10.6882,   1.1062],
        [-10.4787,   1.1675]]), tensor([[ 6.4590,  1.0116],
        [ 6.1157,  0.8197],
        [ 6.2176,  1.0247],
        ...,
        [ 0.1859,  0.6388],
        [-6.0474,  0.8828],
        [-7.5945,  0.9915]]), tensor([[ 0.0775,  0.6315],
        [ 0.7551,  0.6285],
        [ 0.6540,  0.6183],
        ...,
        [ 7.4336,  2.0010],
        [ 0.0142,  1.5505],
        [-2.9611,  1.4544]])

## Graph 4

In [9]:
graph_name = "g4"
drn = False
if drn == True:
    SAVEPATH = os.path.join(DIRECTORY, f"leas_trained_models/drn_{leadtime}/models")
    JSONPATH = os.path.join(DIRECTORY, f"trained_models/drn_{leadtime}/params.json")
else:
    SAVEPATH = os.path.join(DIRECTORY, f"leas_trained_models/sum_stats_{leadtime}/{graph_name}_{leadtime}/models")
    JSONPATH = os.path.join(DIRECTORY, f"leas_trained_models/sum_stats_{leadtime}/{graph_name}_{leadtime}/params.json")


with open(JSONPATH, "r") as f:
    print(f"[INFO] Loading {JSONPATH}")
    args_dict = json.load(f)
config4 = args_dict

[INFO] Loading /home/ltchen/gnnpp/leas_trained_models/sum_stats_24h/g4_24h/params.json


In [10]:
graphs4_train_rf, tests4 = normalize_features_and_create_graphs1(df_train=dataframes['train'], df_valid_test=[dataframes['valid'], dataframes['test_rf'], dataframes['test_f']], station_df=dataframes['stations'], attributes=["dist2", "dist3"], edges=[("dist2", 0.003), ("dist3", 0.0073)], sum_stats = True)
graphs4_valid_rf, graphs4_test_rf, graphs4_test_f = tests4

g4_train_loader = DataLoader(graphs4_train_rf, batch_size=config4['batch_size'], shuffle=True)
g4_valid_loader = DataLoader(graphs4_valid_rf, batch_size=config4['batch_size'], shuffle=True)
g4_test_f_loader = DataLoader(graphs4_test_f, batch_size=config4['batch_size'], shuffle=False)
g4_test_rf_loader = DataLoader(graphs4_test_rf, batch_size=config4['batch_size'], shuffle=False)

train_loader = g4_train_loader
valid_loader = g4_valid_loader
test_f_loader = g4_test_f_loader
test_rf_loader = g4_test_rf_loader
test_loader = [test_f_loader, test_rf_loader]

emb_dim = 20
in_channels = graphs4_train_rf[0].x.shape[1] + emb_dim - 1
edge_dim = graphs4_train_rf[0].num_edge_features
# max_epochs = max_epoch_list[graph_name]
num_nodes = graphs4_train_rf[0].num_nodes

[INFO] Normalizing features...
fit_transform
transform 1
transform 2
transform 3
[INFO] Loading distances from file...
[INFO] Loading distances from file...


100%|██████████| 2612/2612 [00:11<00:00, 217.81it/s]


[INFO] Loading distances from file...
[INFO] Loading distances from file...


100%|██████████| 836/836 [00:03<00:00, 267.04it/s]


[INFO] Loading distances from file...
[INFO] Loading distances from file...


100%|██████████| 732/732 [00:02<00:00, 250.77it/s]


[INFO] Loading distances from file...
[INFO] Loading distances from file...


100%|██████████| 730/730 [00:02<00:00, 252.83it/s]


In [11]:
TRAINNAME = f"{graph_name}_{leadtime}_train_run7"
CKPT_PATH = os.path.join(SAVEPATH, TRAINNAME + '.ckpt')
# RESULTPATH = os.path.join(DIRECTORY, f"leas_trained_models/best_{leadtime}/best_{leadtime}_{graph_name}")
RESULTPATH = JSONPATH = os.path.join(DIRECTORY, f"leas_trained_models/sum_stats_{leadtime}/{graph_name}_{leadtime}")
print(RESULTPATH)

multigraph = Multigraph.load_from_checkpoint(
    CKPT_PATH,
    num_nodes=num_nodes,
    embedding_dim=emb_dim,
    edge_dim=edge_dim,
    in_channels=in_channels,
    hidden_channels_gnn=config4['gnn_hidden'],
    out_channels_gnn=config4['gnn_hidden'],
    num_layers_gnn=config4['gnn_layers'],
    heads=config4['heads'],
    hidden_channels_deepset=config4['gnn_hidden'],
    optimizer_class=AdamW,
    optimizer_params=dict(lr=config4['lr']),
)
multigraph.eval()
trainer = L.Trainer(log_every_n_steps=1, accelerator="gpu", devices=1, enable_progress_bar=True)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


/home/ltchen/gnnpp/leas_trained_models/sum_stats_24h/g4_24h


In [12]:
data_list = ["f", "rf"]
for data, tl in zip(data_list, test_loader):
    preds_list = []
    preds = trainer.predict(model=multigraph, dataloaders=[tl]) # 92 x 976 x 2 forecasts with mu and sigma of 122 stations
    #print(preds[0].shape) # 960 (8*120), 2
    print(preds)
    preds = torch.cat(preds, dim=0)
    preds_list.append(preds)

    targets = dataframes[f"test_{data}"][1]
    print(targets)
    targets = torch.tensor(targets.t2m.values) - 273.15
    print(targets)
    stacked = torch.stack(preds_list)
    final_preds = torch.mean(stacked, dim=0)

    res = multigraph.loss_fn.crps(final_preds, targets)
    print("#############################################")
    print("#############################################")
    print(f"final crps: {res.item()}")
    print("#############################################")
    print("#############################################")

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/home/ltchen/.conda/envs/gnn_env4/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Predicting DataLoader 0: 100%|██████████| 92/92 [00:00<00:00, 187.79it/s]
[tensor([[  4.7082,   0.6821],
        [  2.1756,   0.7980],
        [  5.8617,   0.6912],
        ...,
        [ -1.3311,   1.6847],
        [-11.0604,   1.9382],
        [ -9.7005,   2.4840]]), tensor([[  6.5150,   0.7321],
        [  5.5987,   0.6940],
        [  5.2085,   0.7128],
        ...,
        [ -5.5400,   1.0596],
        [-14.6035,   1.7608],
        [-16.2379,   2.4965]]), tensor([[ -5.3989,   1.8813],
        [ -2.5758,   1.2968],
        [ -2.1234,   1.2648],
        ...,
        [ -8.1959,   2.5269],
        [-14.3307,   2.3926],
        [-16.0313,   3.1749]]), tensor([[-2.8364,  0.9672],
        [-2.6169,  0.9666],
        [-1.5722,  0.9414],
        ...,
        [ 5.7707,  2.4761],
        [ 1.2213,  1.3671],
        [-1.5899,  2.1882]]), tensor([[ 8.5122,  0.7602],
        [ 9.1081,  0.7369],
        [ 6.5058,  0.8328],
        ...,
        [ 0.2916,  2.0116],
        [-6.1655,  1.6279],
    

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


#############################################
#############################################
final crps: 0.6448219898446098
#############################################
#############################################


/home/ltchen/.conda/envs/gnn_env4/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Predicting DataLoader 0: 100%|██████████| 92/92 [00:00<00:00, 198.98it/s]
[tensor([[ 6.8806,  0.6572],
        [ 7.6692,  0.6359],
        [ 7.2862,  0.6736],
        ...,
        [-2.0620,  1.8275],
        [-4.8624,  1.6268],
        [-8.1724,  2.6373]]), tensor([[-2.2594,  0.8516],
        [-3.0868,  0.8658],
        [-2.1363,  0.8642],
        ...,
        [ 2.4070,  1.6024],
        [-0.6290,  1.0541],
        [-2.5588,  1.7091]]), tensor([[-3.0245e-03,  1.5176e+00],
        [ 1.1163e+00,  1.0157e+00],
        [ 1.6052e+00,  1.2164e+00],
        ...,
        [-4.1931e+00,  1.0457e+00],
        [-1.1707e+01,  1.4276e+00],
        [-1.0921e+01,  1.9831e+00]]), tensor([[ 6.3592,  0.6875],
        [ 5.8125,  0.6631],
        [ 6.4384,  0.7172],
        ...,
        [ 0.7136,  0.7687],
        [-7.6512,  0.9822],
        [-9.3595,  1.6906]]), tensor([[ 0.4531,  0.7174],
        [ 0.9443,  0.6908],
        [ 0.9646,  0.7678],
        ...,
        [ 6.2892,  2.0375],
        [-1.0134,  1

## Graph 5

In [13]:
graph_name = "g5"
drn = False
if drn == True:
    SAVEPATH = os.path.join(DIRECTORY, f"leas_trained_models/drn_{leadtime}/models")
    JSONPATH = os.path.join(DIRECTORY, f"trained_models/drn_{leadtime}/params.json")
else:
    SAVEPATH = os.path.join(DIRECTORY, f"leas_trained_models/sum_stats_{leadtime}/{graph_name}_{leadtime}/models")
    JSONPATH = os.path.join(DIRECTORY, f"leas_trained_models/sum_stats_{leadtime}/{graph_name}_{leadtime}/params.json")


with open(JSONPATH, "r") as f:
    print(f"[INFO] Loading {JSONPATH}")
    args_dict = json.load(f)
config5 = args_dict

[INFO] Loading /home/ltchen/gnnpp/leas_trained_models/sum_stats_24h/g5_24h/params.json


In [14]:
graphs5_train_rf, tests5 = normalize_features_and_create_graphs1(df_train=dataframes['train'], df_valid_test=[dataframes['valid'], dataframes['test_rf'], dataframes['test_f']], station_df=dataframes['stations'], attributes=["geo", "alt", "lon", "lat", "alt-orog", "dist2", "dist3"],
                                                                     edges=[("geo", 50),("alt", 4), ("alt-orog", 1.5), ("dist2", 0.003), ("dist3", 0.0074)], sum_stats = True)
graphs5_valid_rf, graphs5_test_rf, graphs5_test_f = tests5

g5_train_loader = DataLoader(graphs5_train_rf, batch_size=config5['batch_size'], shuffle=True)
g5_valid_loader = DataLoader(graphs5_valid_rf, batch_size=config5['batch_size'], shuffle=True)
g5_test_f_loader = DataLoader(graphs5_test_f, batch_size=config5['batch_size'], shuffle=False)
g5_test_rf_loader = DataLoader(graphs5_test_rf, batch_size=config5['batch_size'], shuffle=False)

train_loader = g5_train_loader
valid_loader = g5_valid_loader
test_f_loader = g5_test_f_loader
test_rf_loader = g5_test_rf_loader
test_loader = [test_f_loader, test_rf_loader]

emb_dim = 20
in_channels = graphs5_train_rf[0].x.shape[1] + emb_dim - 1
edge_dim = graphs5_train_rf[0].num_edge_features
# max_epochs = max_epoch_list[graph_name]
num_nodes = graphs5_train_rf[0].num_nodes

[INFO] Normalizing features...
fit_transform
transform 1
transform 2
transform 3
[INFO] Loading distances from file...
[INFO] Loading distances from file...


100%|██████████| 2612/2612 [00:12<00:00, 207.96it/s]


[INFO] Loading distances from file...
[INFO] Loading distances from file...


100%|██████████| 836/836 [00:03<00:00, 269.94it/s]


[INFO] Loading distances from file...
[INFO] Loading distances from file...


100%|██████████| 732/732 [00:02<00:00, 262.97it/s]


[INFO] Loading distances from file...
[INFO] Loading distances from file...


100%|██████████| 730/730 [00:03<00:00, 239.95it/s]


In [16]:
TRAINNAME = f"{graph_name}_{leadtime}_train_run7"
CKPT_PATH = os.path.join(SAVEPATH, TRAINNAME + '.ckpt')
# RESULTPATH = os.path.join(DIRECTORY, f"leas_trained_models/best_{leadtime}/best_{leadtime}_{graph_name}")
RESULTPATH = JSONPATH = os.path.join(DIRECTORY, f"leas_trained_models/sum_stats_{leadtime}/{graph_name}_{leadtime}")
print(RESULTPATH)

multigraph = Multigraph.load_from_checkpoint(
    CKPT_PATH,
    num_nodes=num_nodes,
    embedding_dim=emb_dim,
    edge_dim=edge_dim,
    in_channels=in_channels,
    hidden_channels_gnn=config5['gnn_hidden'],
    out_channels_gnn=config5['gnn_hidden'],
    num_layers_gnn=config5['gnn_layers'],
    heads=config5['heads'],
    hidden_channels_deepset=config5['gnn_hidden'],
    optimizer_class=AdamW,
    optimizer_params=dict(lr=config5['lr']),
)
multigraph.eval()
trainer = L.Trainer(log_every_n_steps=1, accelerator="gpu", devices=1, enable_progress_bar=True)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


/home/ltchen/gnnpp/leas_trained_models/sum_stats_24h/g5_24h


In [17]:
data_list = ["f", "rf"]
for data, tl in zip(data_list, test_loader):
    preds_list = []
    preds = trainer.predict(model=multigraph, dataloaders=[tl]) # 92 x 976 x 2 forecasts with mu and sigma of 122 stations
    #print(preds[0].shape) # 960 (8*120), 2
    print(preds)
    preds = torch.cat(preds, dim=0)
    preds_list.append(preds)

    targets = dataframes[f"test_{data}"][1]
    print(targets)
    targets = torch.tensor(targets.t2m.values) - 273.15
    print(targets)
    stacked = torch.stack(preds_list)
    final_preds = torch.mean(stacked, dim=0)

    res = multigraph.loss_fn.crps(final_preds, targets)
    print("#############################################")
    print("#############################################")
    print(f"final crps: {res.item()}")
    print("#############################################")
    print("#############################################")

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/home/ltchen/.conda/envs/gnn_env4/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Predicting DataLoader 0: 100%|██████████| 92/92 [00:00<00:00, 117.96it/s]
[tensor([[  4.1950,   0.9911],
        [  1.7697,   0.8710],
        [  4.9973,   0.8300],
        ...,
        [ -0.9208,   1.7580],
        [-10.0745,   1.8337],
        [-11.1108,   1.9267]]), tensor([[  5.5671,   0.9001],
        [  5.2626,   0.8265],
        [  5.5094,   0.9141],
        ...,
        [ -4.8573,   0.9826],
        [-13.8841,   1.2844],
        [-14.1744,   1.4031]]), tensor([[ -2.4596,   1.5348],
        [ -3.4307,   1.5463],
        [ -0.9770,   1.4141],
        ...,
        [ -6.5834,   1.8683],
        [-14.1063,   1.9210],
        [-15.7810,   2.1399]]), tensor([[-2.0096,  0.9755],
        [-2.8760,  1.1189],
        [-1.5275,  0.9461],
        ...,
        [ 5.7045,  2.3250],
        [ 0.5828,  1.4854],
        [-1.4527,  1.3349]]), tensor([[ 8.3189,  0.8856],
        [ 9.0644,  0.8546],
        [ 7.2373,  0.8515],
        ...,
        [ 1.0976,  1.8648],
        [-6.5208,  1.6840],
    

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


            time  station_id     t2m
0     2017-01-01           0  278.65
1     2017-01-01           1  275.25
2     2017-01-01           2  279.75
3     2017-01-01           3  279.15
4     2017-01-01           4  275.05
...          ...         ...     ...
87595 2018-12-31         115  277.95
87596 2018-12-31         116  276.85
87597 2018-12-31         117  276.35
87598 2018-12-31         118  270.65
87599 2018-12-31         119  268.85

[87600 rows x 3 columns]
tensor([ 5.5000,  2.1000,  6.6000,  ...,  3.2000, -2.5000, -4.3000],
       dtype=torch.float64)
#############################################
#############################################
final crps: 0.6357459247678992
#############################################
#############################################


/home/ltchen/.conda/envs/gnn_env4/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Predicting DataLoader 0: 100%|██████████| 92/92 [00:00<00:00, 118.99it/s]
[tensor([[ 7.2016,  0.6778],
        [ 7.6714,  0.6955],
        [ 7.1533,  0.6836],
        ...,
        [-2.7240,  1.5830],
        [-5.8708,  1.9189],
        [-8.8077,  1.9508]]), tensor([[-2.3624,  0.8605],
        [-3.2567,  1.0314],
        [-2.3407,  0.8751],
        ...,
        [ 2.6476,  1.3453],
        [-1.5183,  1.0500],
        [-2.2734,  1.0608]]), tensor([[  1.8400,   1.4942],
        [  1.9070,   1.4122],
        [  2.8856,   1.5210],
        ...,
        [ -4.0394,   0.9806],
        [-11.4361,   1.2045],
        [-11.6444,   1.3054]]), tensor([[ 5.9642,  0.7385],
        [ 6.3269,  0.7293],
        [ 6.3401,  0.7205],
        ...,
        [ 0.8430,  0.8532],
        [-6.4399,  1.2617],
        [-8.1803,  1.4579]]), tensor([[ 0.6744,  0.7218],
        [ 0.7551,  0.6912],
        [ 1.5113,  0.7317],
        ...,
        [ 6.2545,  2.4164],
        [-0.7488,  1.6463],
        [-3.6015,  1.5424]])